In [1]:
from pdf2image import convert_from_path
import pytesseract
import os
import time
import re
import json
import pandas as pd
from dotenv import load_dotenv
from openai import OpenAI

# (1) 환경 변수 로드
load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

results = []

# (2) test_01.pdf ~ test_10.pdf 순차 처리
for i in range(1, 11):
    pdf_path = f"/home/ys0660/JDProject/sample/test_{i:02d}.pdf"
    if not os.path.exists(pdf_path):
        print(f"⚠️ 파일 없음: {pdf_path}")
        continue

    print(f"\n📄 처리 중: {pdf_path}")

    try:
        # (3) PDF의 앞 2페이지만 이미지로 변환
        pages = convert_from_path(pdf_path, first_page=1, last_page=2)

        # (4) OCR로 텍스트 추출
        text = ""
        for j, page in enumerate(pages):
            page_text = pytesseract.image_to_string(page, lang="kor+eng")
            text += f"\n--- Page {j+1} ---\n" + page_text
            page.close()  # 이미지 메모리 해제
        del pages

        # (5) 프롬프트 (수정 금지)
        prompt = f"""
        You are a patent information extraction assistant.
        From the following patent text, extract the following fields and return them **only** as a valid JSON object:

        - country: The country name or code (e.g., "US", "KR")
        - date: The publication or filing date in ISO format (YYYY-MM-DD)
        - title: The name of the invention or patent
        - applicant: The applicant or organization that submitted the patent
        - ipc_code: A list of IPC classification codes (e.g., ["A61K 39/05", "C07K 14/195"])
        - naics_code: A list of estimated NAICS codes relevant to the invention's industry (e.g., ["325412", "334413"])
        - abstract: The abstract text

        ### Additional Rules:
        1. For IPC codes:
           - Extract from the text, then verify semantic consistency with the abstract.
           - If invalid or irrelevant, replace with valid and relevant IPC codes (standard format required).
        2. For NAICS codes:
           - Infer the most relevant NAICS industry classification based on the abstract and title.
           - Return one or more 6-digit NAICS codes that best match the invention’s domain.
           - If uncertain, return `null`.

        If some information is missing, return `null` for that field.

        Patent text:
        {text}
        """

        # (6) GPT 호출
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "You extract structured patent metadata in JSON."},
                {"role": "user", "content": prompt},
            ],
            temperature=0
        )

        response_text = response.choices[0].message.content.strip()

        # (7) JSON 블록 추출
        match = re.search(r"\{.*\}", response_text, re.DOTALL)
        if not match:
            print(f"❌ JSON 블록을 찾을 수 없습니다: {pdf_path}")
            continue

        try:
            data = json.loads(match.group(0))
            data["file_name"] = os.path.basename(pdf_path)
            results.append(data)
            print(f"✅ 완료: {pdf_path}")
        except json.JSONDecodeError:
            print(f"❌ JSON 파싱 실패: {pdf_path}")
            continue

    except Exception as e:
        print(f"❌ 오류 발생 ({pdf_path}): {e}")
        continue

    # (8) 각 파일 처리 후 3초 대기
    time.sleep(3)

# (9) CSV 저장
if results:
    df = pd.DataFrame(results)
    df["ipc_code"] = df["ipc_code"].apply(lambda x: ", ".join(x) if isinstance(x, list) else x)
    df["naics_code"] = df["naics_code"].apply(lambda x: ", ".join(x) if isinstance(x, list) else x)

    df.to_csv("user_patent_info_all_stable.csv", index=False, encoding="utf-8-sig")
    print("\n✅ CSV 저장 완료: user_patent_info_all_stable.csv")
else:
    print("\n⚠️ 처리된 결과가 없습니다.")



📄 처리 중: /home/ys0660/JDProject/sample/test_01.pdf
✅ 완료: /home/ys0660/JDProject/sample/test_01.pdf

📄 처리 중: /home/ys0660/JDProject/sample/test_02.pdf
✅ 완료: /home/ys0660/JDProject/sample/test_02.pdf

📄 처리 중: /home/ys0660/JDProject/sample/test_03.pdf
✅ 완료: /home/ys0660/JDProject/sample/test_03.pdf

📄 처리 중: /home/ys0660/JDProject/sample/test_04.pdf
✅ 완료: /home/ys0660/JDProject/sample/test_04.pdf

📄 처리 중: /home/ys0660/JDProject/sample/test_05.pdf
✅ 완료: /home/ys0660/JDProject/sample/test_05.pdf

📄 처리 중: /home/ys0660/JDProject/sample/test_06.pdf
✅ 완료: /home/ys0660/JDProject/sample/test_06.pdf

📄 처리 중: /home/ys0660/JDProject/sample/test_07.pdf
✅ 완료: /home/ys0660/JDProject/sample/test_07.pdf

📄 처리 중: /home/ys0660/JDProject/sample/test_08.pdf
✅ 완료: /home/ys0660/JDProject/sample/test_08.pdf

📄 처리 중: /home/ys0660/JDProject/sample/test_09.pdf
✅ 완료: /home/ys0660/JDProject/sample/test_09.pdf

📄 처리 중: /home/ys0660/JDProject/sample/test_10.pdf
✅ 완료: /home/ys0660/JDProject/sample/test_10.pdf

✅ CSV 저장 

In [5]:
from pdf2image import convert_from_path
from PIL import Image
import pytesseract
import os
from dotenv import load_dotenv
from openai import OpenAI
import json
import re
import pandas as pd

# 환경 변수 로드
load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

results = []

def detect_patent_language(page):
    """특허 상단부 텍스트 기반으로 언어 감지"""
    width, height = page.size
    header_region = page.crop((0, 0, width, height * 0.40))
    header_text = pytesseract.image_to_string(header_region, lang="kor+eng").lower()

    if any(kw in header_text for kw in ["대한민국특허청", "korean intellectual property", "KR", "공개일자", "등록", "출원"]):
        return "ko"
    elif any(kw in header_text for kw in ["United States Patent", "United States", "Patent"]):
        return "en"
    else:
        # 기본값: 영어로
        return "en"

# PDF 반복
for i in range(1, 11):
    pdf_path = f"/home/ys0660/JDProject/sample/test_{i:02d}.pdf"
    if not os.path.exists(pdf_path):
        print(f"⚠️ 파일이 존재하지 않습니다: {pdf_path}")
        continue

    print(f"\n📄 처리 중: {pdf_path}")

    pages = convert_from_path(pdf_path, first_page=1, last_page=2)

    # (1) 첫 페이지에서 언어/국가 판단
    patent_lang = detect_patent_language(pages[0])
    print(f"🌐 특허 언어 판단: {patent_lang}")

    text = ""
    custom_config = "--psm 6"

    # (2) 언어별 OCR 방식
    for j, page in enumerate(pages):
        width, height = page.size

        if patent_lang == "ko":
            # ✅ 한국어 → 모든 부분 1단 OCR (명칭+요약 포함)
            page_text = pytesseract.image_to_string(page, lang="kor+eng", config=custom_config)
        else:
            # ✅ 영어 → 2단 OCR (column-aware)
            mid_x = width // 2
            left_col = page.crop((0, 0, mid_x, height))
            right_col = page.crop((mid_x, 0, width, height))

            left_text = pytesseract.image_to_string(left_col, lang="kor+eng", config=custom_config)
            right_text = pytesseract.image_to_string(right_col, lang="kor+eng", config=custom_config)
            page_text = left_text.strip() + "\n" + right_text.strip()

        text += f"\n--- Page {j+1} ---\n{page_text.strip()}\n"

    # (3) GPT 프롬프트
    prompt = f"""
You are a patent information extraction assistant.
From the following patent text, extract the following fields and return them **only** as a valid JSON object:

- country: The country name or code (e.g., "US", "KR")
- date: The **filing date** in ISO format (YYYY-MM-DD)
- title: The title of the invention or patent
- applicant: The applicant or organization that filed the patent
- ipc_code: A list of IPC classification codes (e.g., ["A61K 39/05", "C07K 14/195"])
- naics_code: A list of up to **three (3)** estimated NAICS codes relevant to the invention’s industry (e.g., ["325412", "334413", "541715"])
- abstract: The full abstract text (including text continued on the next page)

### Extraction Rules:

1. **Filing Date (출원일자)**:
   - Use the field labeled **(22) PCT Filed**, **(22) Application Filed**, or similar.
   - **Do not use (43) Publication Date** or **(45) Published** — these are *not* filing dates.
   - Return in ISO format (YYYY-MM-DD). If no valid filing date exists, return `null`.

2. **Applicant (출원인 / 특허권자)**:
   - Extract from field **(73) Assignee / Applicant / Patentee**.
   - If multiple applicants exist, return as a comma-separated string.

3. **IPC Codes**:
   - Extract all IPC classification codes **exactly as they appear** in the text (e.g., under (51) Int. Cl.).
   - Verify that each IPC code follows valid IPC syntax (e.g., “A61K 39/05”).
   - If a code is invalid or malformed, replace it with the **closest semantically correct and valid IPC code**.
   - Do **not invent new codes** that are not in the text.

4. **NAICS Codes**:
   - Infer the **three most relevant 6-digit NAICS codes** based on the title and abstract.
   - Select codes representing the main industrial domains or applications.
   - If uncertain, return `null`.

5. **Abstract (요약문)**:
   - Combine all parts of the abstract, even if marked as **“(continued)”** or split across pages.
   - Return the **entire abstract text**, preserving readability and continuity.

6. **Missing Information**:
   - If any field cannot be found, return `null` for that field.

Patent text:
{text}
"""

    # (4) GPT 호출
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You extract structured patent metadata in JSON."},
            {"role": "user", "content": prompt},
        ],
        temperature=0
    )

    response_text = response.choices[0].message.content.strip()

    # (5) JSON 파싱
    match = re.search(r"\{.*\}", response_text, re.DOTALL)
    if not match:
        print(f"❌ JSON 블록을 찾을 수 없습니다: {pdf_path}")
        continue

    try:
        data = json.loads(match.group(0))
        data["file_name"] = os.path.basename(pdf_path)
        data["ocr_text"] = text.strip()
        results.append(data)
    except json.JSONDecodeError:
        print(f"❌ JSON 파싱 실패: {pdf_path}")
        continue

# (6) CSV 저장
if results:
    df = pd.DataFrame(results)
    df["ipc_code"] = df["ipc_code"].apply(lambda x: ", ".join(x) if isinstance(x, list) else x)
    df["naics_code"] = df["naics_code"].apply(lambda x: ", ".join(x) if isinstance(x, list) else x)
    df.to_csv("user_patent_info_all.csv", index=False, encoding="utf-8-sig")
    print("✅ CSV 저장 완료: user_patent_info_all.csv (패턴기반 언어감지 적용)")
else:
    print("⚠️ 처리된 결과가 없습니다.")



📄 처리 중: /home/ys0660/JDProject/sample/test_01.pdf
🌐 특허 언어 판단: en

📄 처리 중: /home/ys0660/JDProject/sample/test_02.pdf
🌐 특허 언어 판단: en

📄 처리 중: /home/ys0660/JDProject/sample/test_03.pdf
🌐 특허 언어 판단: en

📄 처리 중: /home/ys0660/JDProject/sample/test_04.pdf
🌐 특허 언어 판단: ko

📄 처리 중: /home/ys0660/JDProject/sample/test_05.pdf
🌐 특허 언어 판단: ko

📄 처리 중: /home/ys0660/JDProject/sample/test_06.pdf
🌐 특허 언어 판단: en

📄 처리 중: /home/ys0660/JDProject/sample/test_07.pdf
🌐 특허 언어 판단: ko

📄 처리 중: /home/ys0660/JDProject/sample/test_08.pdf
🌐 특허 언어 판단: ko

📄 처리 중: /home/ys0660/JDProject/sample/test_09.pdf
🌐 특허 언어 판단: en

📄 처리 중: /home/ys0660/JDProject/sample/test_10.pdf
🌐 특허 언어 판단: en
✅ CSV 저장 완료: user_patent_info_all.csv (패턴기반 언어감지 적용)


In [12]:
import pdfplumber
from pdf2image import convert_from_path
import pytesseract
import os
from dotenv import load_dotenv
from openai import OpenAI
import json
import re
import pandas as pd

# 환경 변수 로드
load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

results = []

def extract_text_from_pdf(pdf_path):
    """텍스트 기반 PDF → pdfplumber로 추출, 비어 있으면 OCR fallback"""
    text = ""

    try:
        with pdfplumber.open(pdf_path) as pdf:
            for i, page in enumerate(pdf.pages[:2]):  # 앞 2페이지
                extracted = page.extract_text()
                if extracted and extracted.strip():
                    text += f"\n--- Page {i+1} (text) ---\n" + extracted
                else:
                    # OCR fallback (이미지화 후 pytesseract)
                    image = page.to_image(resolution=300).original
                    ocr_text = pytesseract.image_to_string(image, lang="kor+eng")
                    text += f"\n--- Page {i+1} (ocr) ---\n" + ocr_text
    except Exception as e:
        print(f"⚠️ pdfplumber 실패, OCR 전체 처리로 전환: {e}")
        pages = convert_from_path(pdf_path, first_page=1, last_page=2)
        for j, page in enumerate(pages):
            ocr_text = pytesseract.image_to_string(page, lang="kor+eng")
            text += f"\n--- Page {j+1} (ocr only) ---\n" + ocr_text
    return text.strip()

# PDF 반복
for i in range(1, 11):
    pdf_path = f"/home/ys0660/JDProject/sample/test_{i:02d}.pdf"
    if not os.path.exists(pdf_path):
        print(f"⚠️ 파일이 존재하지 않습니다: {pdf_path}")
        continue

    print(f"\n📄 처리 중: {pdf_path}")
    text = extract_text_from_pdf(pdf_path)

    # GPT 프롬프트
    prompt = f"""
You are a patent information extraction assistant.
From the following patent text, extract the following fields and return them **only** as a valid JSON object:

- country: The country name or code (e.g., "US", "KR")
- date: The **filing date** in ISO format (YYYY-MM-DD)
- title: The title of the invention or patent
- applicant: The applicant or organization that filed the patent
- ipc_code: A list of IPC classification codes (e.g., ["A61K 39/05", "C07K 14/195"])
- naics_code: A list of up to **three (3)** estimated NAICS codes relevant to the invention’s industry (e.g., ["325412", "334413", "541715"])
- abstract: The full abstract text (including text continued on the next page)

### Extraction Rules:

1. **Filing Date (출원일자)**:
   - Use the field labeled **(22) PCT Filed**, **(22) Application Filed**, or similar.
   - **Do not use (43) Publication Date** or **(45) Published** — these are *not* filing dates.
   - Return in ISO format (YYYY-MM-DD). If no valid filing date exists, return `null`.

2. **Applicant (출원인 / 특허권자)**:
   - Extract from field **(73) Assignee / Applicant / Patentee**.
   - If multiple applicants exist, return as a comma-separated string.

3. **IPC Codes**:
   - Extract all IPC classification codes **exactly as they appear** in the text (e.g., under (51) Int. Cl.).
   - Verify that each IPC code follows valid IPC syntax (e.g., “A61K 39/05”).
   - If a code is invalid or malformed, replace it with the **closest semantically correct and valid IPC code**.
   - Do **not invent new codes** that are not in the text.

4. **NAICS Codes**:
   - Infer the **three most relevant 6-digit NAICS codes** based on the title and abstract.
   - Select codes representing the main industrial domains or applications.
   - If uncertain, return `null`.

5. **Abstract (요약문)**:
   - Combine all parts of the abstract, even if marked as **“(continued)”** or split across pages.
   - Return the **entire abstract text**, preserving readability and continuity.

6. **Missing Information**:
   - If any field cannot be found, return `null` for that field.

Patent text:
{text}
"""

    # GPT 호출
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You extract structured patent metadata in JSON."},
            {"role": "user", "content": prompt},
        ],
        temperature=0
    )

    response_text = response.choices[0].message.content.strip()

    # JSON 파싱
    match = re.search(r"\{.*\}", response_text, re.DOTALL)
    if not match:
        print(f"❌ JSON 블록을 찾을 수 없습니다: {pdf_path}")
        continue

    try:
        data = json.loads(match.group(0))
        data["file_name"] = os.path.basename(pdf_path)
        data["ocr_text"] = text
        results.append(data)
    except json.JSONDecodeError:
        print(f"❌ JSON 파싱 실패: {pdf_path}")
        continue

# CSV 저장
if results:
    df = pd.DataFrame(results)
    df["ipc_code"] = df["ipc_code"].apply(lambda x: ", ".join(x) if isinstance(x, list) else x)
    df["naics_code"] = df["naics_code"].apply(lambda x: ", ".join(x) if isinstance(x, list) else x)
    df.to_csv("user_patent_info_all_v5.csv", index=False, encoding="utf-8-sig")
    print("✅ CSV 저장 완료: user_patent_info_all_v5.csv (pdfplumber 기반)")
else:
    print("⚠️ 처리된 결과가 없습니다.")



📄 처리 중: /home/ys0660/JDProject/sample/test_01.pdf

📄 처리 중: /home/ys0660/JDProject/sample/test_02.pdf

📄 처리 중: /home/ys0660/JDProject/sample/test_03.pdf

📄 처리 중: /home/ys0660/JDProject/sample/test_04.pdf

📄 처리 중: /home/ys0660/JDProject/sample/test_05.pdf

📄 처리 중: /home/ys0660/JDProject/sample/test_06.pdf

📄 처리 중: /home/ys0660/JDProject/sample/test_07.pdf

📄 처리 중: /home/ys0660/JDProject/sample/test_08.pdf

📄 처리 중: /home/ys0660/JDProject/sample/test_09.pdf

📄 처리 중: /home/ys0660/JDProject/sample/test_10.pdf
✅ CSV 저장 완료: user_patent_info_all_v5.csv (pdfplumber 기반)
